# Imports

In [1]:
import os
import warnings
warnings.filterwarnings("ignore")

from typing import Dict, List, Tuple

import numpy as np
import pandas as pd
import scipy as sp
import scipy.signal
import scipy.stats
from scipy.stats.mstats import gmean

from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV, LeaveOneOut, StratifiedKFold
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from scipy.stats import mode
from sklearn.model_selection import StratifiedKFold, GridSearchCV, LeaveOneOut

from lightgbm import LGBMClassifier
from xgboost import XGBClassifier

from utils import Dataset, variance_thresholding, standardize, mcc, calculate_metrics, calculate_metrics_statistics

In [2]:
# parameters for Welch's method for estimating power spectrum

NPERSEG = 11                    # length of segment
NOVERLAP = int(0.75 * NPERSEG)  # overlap of segments
NFFT = NPERSEG                  # length of FFT
WINDOW = "hann"                 # window function type

# parameters for saving data
PROCESSED_DATA_DIR = "processed_data24h"
DEPRESJON_PREFIX = "manual_depresjon24h"
HYPERAKTIV_PREFIX = "manual_hyperaktiv24h"
PSYKOSE_PREFIX = "manual_psykose24h"

# Manual feature extraction

## Helper functions

In [3]:
def basic_data_cleaning(data: List[pd.DataFrame]) -> List[pd.DataFrame]:
    """
    Assumes DataFrames with "timestamp", "date" and "activity" columns.
    
    Performs cleaning operations:
    - assure format YYYY-MM-DD HH:MM:SS for "timestamp"
    - drop redundant "date" column
    - assure float32 format for "activity"
    
    :param data: list of DataFrames
    :returns: list of cleaned DataFrames
    """
    data = [df.copy() for df in data]  # create copy to avoid side effects
    
    for df in data:
        df["timestamp"] = pd.to_datetime(df["timestamp"],
                                         format="%Y-%m-%d %H:%M:%S")
        df.drop("date", axis=1, inplace=True)
        df["activity"] = df["activity"].astype(np.float32)
    
    return data


def get_day_part(df: pd.DataFrame, part: str) -> pd.DataFrame:
    """
    For given DataFrame with "timestamp" column returns only those rows that
    correspond to the chosen part of day.
    
    Parts are "day" and "night", defined as:
    - "day": [8:00, 21:00)
    - "night": [21:00, 8:00)
    
    :param df: DataFrame to select rows from
    :param part: part of day, either "day" or "night"
    :returns: DataFrame, subset of rows of df
    """
    if part == "day":
        df = df.loc[(df["timestamp"].dt.hour >= 8) &
                    (df["timestamp"].dt.hour < 21)]
    elif part == "night":
        df = df.loc[(df["timestamp"].dt.hour >= 21) |
                    (df["timestamp"].dt.hour < 8)]
    else:
        raise ValueError(f'Part should be "day" or "night", got "{part}"')
        
    return df


def fill_missing_activity(df: pd.DataFrame) -> pd.DataFrame:
    """
    Makes sure that "timestamp" column has minute resolution with no missing
    values from start to end and replaces all NaNs in "activity" column with
    mean average value.
    
    :param data: DataFrame with "timestamp" and "activity" columns
    :returns: cleaned DataFrame
    """
    df = df.copy()  # create copy to avoid side effects
    
    # resample to the basic frequency, i.e. minute; this will create NaNs for
    # any rows that may be missing
    df = df.resample("min", on="timestamp").mean()
    
    # recreate index and "timestamp" column
    df = df.reset_index()
    
    # fill any NaNs with mean activity value
    df["activity"] = df["activity"].fillna(df["activity"].mean())

    return df


def resample(df: pd.DataFrame, freq: str = "H") -> pd.DataFrame:
    """
    Resamples time series DataFrame with given frequency, aggregating each
    segment with a mean.

    :param df: DataFrame with "timestamp" and "activity" columns
    :param freq: resampling frequency passed to Pandas resample() function
    :returns: DataFrame with "timestamp" and "activity" columns
    """
    df = df.copy()  # create copy to avoid side effects
    
    # group with given frequency
    df = df.resample(freq, on="timestamp").mean()

    # recreate "timestamp" column
    df = df.reset_index()

    return df


def proportion_of_zeros(x: np.ndarray) -> float:
    """
    Calculates proportion of zeros in given array, i.e. number of zeros divided
    by length of array.
    
    :param x: 1D Numpy array
    :returns: proportion of zeros
    """
    # we may be dealing with floating numbers, we can't use direct comparison
    zeros_count = np.sum(np.isclose(x, 0))
    return zeros_count / len(x)


def power_spectral_density(df: pd.DataFrame) -> np.ndarray:
    """
    Calculates power spectral density (PSD) from "activity" column of a
    DataFrame.
    
    :param df: DataFrame with "activity" column
    :returns: 1D Numpy array with power spectral density
    """
    psd = scipy.signal.welch(
        x=df["activity"].values,
        fs=(1/11),
        nperseg=11,
        noverlap=10,
        nfft=NFFT,
        window=WINDOW,
        scaling="density"
    )[1]
    return psd


def spectral_flatness(df: pd.DataFrame) -> float:
    """
    Calculates spectral flatness of a signal, i.e. a geometric mean of the
    power spectrum divided by the arithmetic mean of the power spectrum.
    
    If some frequency bins in the power spectrum are close to zero, they are
    removed prior to calculation of spectral flatness to avoid calculation of
    log(0).
    
    :param df: DataFrame with "activity" column
    :returns: spectral flatness value
    """
    power_spectrum = scipy.signal.welch(
        df["activity"].values,
        fs=(1/11),
        nperseg=11,
        noverlap=10,
        nfft=NFFT,
        window=WINDOW,
        scaling="spectrum"
    )[1]
    
    non_zeros_mask = ~np.isclose(power_spectrum, 0)
    power_spectrum = power_spectrum[non_zeros_mask]
    
    return scipy.stats.gmean(power_spectrum) / power_spectrum.mean()

## Feature extraction

In [4]:
def extract_time_features(df: pd.DataFrame) -> pd.DataFrame:
    """
    Extracts features from activity signal in time domain.
    
    :param df_resampled: DataFrame with "activity" column
    :returns: DataFrame with a single row representing features
    """
    X = df["activity"].values
    
    features = {
        "minimum": np.min(X),
        "maximum": np.max(X),
        "mean": np.mean(X),
        "median": np.median(X),
        "variance": np.var(X, ddof=1),  # apply Bessel's correction
        "kurtosis": sp.stats.kurtosis(X),
        "skewness": sp.stats.skew(X),
        "coeff_of_var": sp.stats.variation(X),
        "iqr": sp.stats.iqr(X),
        "trimmed_mean": sp.stats.trim_mean(X, proportiontocut=0.1),
        "entropy": sp.stats.entropy(X, base=2),
        "proportion_of_zeros": proportion_of_zeros(X)
    }
    
    return pd.DataFrame([features])

In [5]:
def extract_frequency_features(df: pd.DataFrame) -> pd.DataFrame:
    """
    Extracts features from activity signal in frequency domain, i.e. calculated
    from its Power Spectral Density (PSD).
    
    :param df: DataFrame with "activity" column
    :returns: DataFrame with a single row representing features
    """
    X = power_spectral_density(df)
    
    features = {
        "minimum": np.min(X),
        "maximum": np.max(X),
        "mean": np.mean(X),
        "median": np.median(X),
        "variance": np.var(X),
        "kurtosis": sp.stats.kurtosis(X),
        "skewness": sp.stats.skew(X),
        "coeff_of_var": sp.stats.variation(X),
        "iqr": sp.stats.iqr(X),
        "trimmed_mean": sp.stats.trim_mean(X, proportiontocut=0.1),
        "entropy": sp.stats.entropy(X, base=2),
        "spectral_flatness": spectral_flatness(df)
    }
    
    return pd.DataFrame([features])

In [6]:
def extract_features_for_dataframes(dfs: List[pd.DataFrame], freq: str = "H") \
        -> Dict[str, pd.DataFrame]:
    """
    Calculates time and frequency features for given DataFrames. Uses given
    frequency for resampling.
    
    Calculates features separately for:
    - nights: [21:00, 8:00)
    
    :param dfs: list of DataFrames to extract features from; each one has to
    have "timestamp" and "activity" columns
    :param freq: resampling frequency
    :returns: dictionary with keys "night", corresponding
    to features from given parts of day
    """
    full_dfs = basic_data_cleaning(dfs)
    full_dfs = [fill_missing_activity(df) for df in full_dfs]
    full_dfs = [resample(df, freq=freq) for df in full_dfs]

    datasets = {}
    
    for part, list_of_dfs in [("night", full_dfs)]:
        features = []
        for df in list_of_dfs:
            time_features = extract_time_features(df)
            freq_features = extract_frequency_features(df)

            merged_features = pd.merge(
                time_features,
                freq_features,
                left_index=True,
                right_index=True,
                suffixes=["_time", "_freq"]
            )
            features.append(merged_features)

        datasets[part] = pd.concat(features)
        datasets[part].reset_index(drop=True, inplace=True)
    
    return datasets



## Depresjon

In [7]:
dataset = Dataset(dirpath=os.path.join("data_night", "depresjon"))
condition = dataset.condition
control = dataset.control

In [8]:
condition[0]

,timestamp,date,activity
0,2003-05-07 21:00:00,2003-05-07,212
1,2003-05-07 21:01:00,2003-05-07,212
2,2003-05-07 21:02:00,2003-05-07,17
3,2003-05-07 21:03:00,2003-05-07,38
4,2003-05-07 21:04:00,2003-05-07,82
...,...,...,...
655,2003-05-08 07:55:00,2003-05-08,9
656,2003-05-08 07:56:00,2003-05-08,23
657,2003-05-08 07:57:00,2003-05-08,0
658,2003-05-08 07:58:00,2003-05-08,0


In [9]:
condition_parts_dfs = extract_features_for_dataframes(condition, freq="H")
control_parts_dfs = extract_features_for_dataframes(control, freq="H")

datasets = {}

for part in ["night"]:
    condition_df = condition_parts_dfs[part]
    control_df = control_parts_dfs[part]
    
    entire_df = pd.concat([condition_df, control_df], ignore_index=True)
    
    # Przypisujemy wynik do słownika datasets
    datasets[part] = entire_df

In [10]:
for part, df in datasets.items():
    filename = f"{DEPRESJON_PREFIX}_{part}.csv"
    filepath = os.path.join(PROCESSED_DATA_DIR, filename)
    df.to_csv(filepath, index=False)

In [11]:
condition_df

,minimum_time,maximum_time,mean_time,median_time,variance_time,kurtosis_time,skewness_time,coeff_of_var_time,iqr_time,trimmed_mean_time,...,mean_freq,median_freq,variance_freq,kurtosis_freq,skewness_freq,coeff_of_var_freq,iqr_freq,trimmed_mean_freq,entropy_freq,spectral_flatness
0,0.966667,148.733337,25.707579,10.000000,1821.470337,4.635292,2.443026,1.582900,17.500001,14.787038,...,13916.573242,10138.624023,9.799489e+07,1.124900,1.744361,0.711327,1125.172607,13916.574219,2.302669,0.847654
1,2.083333,120.050003,33.522724,12.650000,1455.296509,0.244883,1.169350,1.085026,51.141668,27.401852,...,13156.655273,7906.802734,2.379827e+08,0.439530,1.376406,1.172539,11228.363770,13156.653320,1.720446,0.405417
2,0.583333,178.116669,46.118183,17.783333,3332.064453,0.344395,1.195673,1.193406,74.924999,36.511112,...,39708.550781,24891.986328,1.254300e+09,-0.393184,0.991210,0.891901,37194.058105,39708.550781,2.048502,0.632379
3,0.916667,85.483330,25.483332,10.866667,968.310364,-0.284877,1.166769,1.164273,28.716667,21.546295,...,8917.659180,6521.940918,5.759565e+07,-0.566328,0.864708,0.851028,9563.873108,8917.659180,2.086632,0.667920
4,2.666667,111.233330,24.503029,9.816667,1099.974243,2.282830,1.835684,1.290551,20.741667,17.292593,...,9094.897461,6941.023438,4.072176e+07,0.514084,1.383717,0.701642,3578.424805,9094.897461,2.283462,0.816446
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
377,0.000000,16.533333,2.636364,0.000000,27.489933,2.537389,1.916340,1.896204,2.925000,1.385185,...,143.984879,22.406513,3.182891e+04,-1.497008,0.706503,1.239065,283.116976,143.984879,1.490054,0.339778
378,0.000000,1.850000,0.313636,0.000000,0.474545,0.903060,1.684765,2.094191,0.025000,0.177778,...,2.927676,1.562682,5.071979e+00,0.299998,1.385461,0.769247,1.630936,2.927676,2.233446,0.799678
379,0.000000,1.200000,0.160606,0.000000,0.147737,3.049299,2.128762,2.281849,0.000000,0.062963,...,1.182480,0.732170,1.642501e+00,-0.137021,1.096713,1.083825,1.356251,1.182480,1.780805,0.430636
380,0.000000,1.350000,0.228788,0.000000,0.177838,2.642057,1.935837,1.757452,0.275000,0.129630,...,1.330155,1.300855,9.277220e-01,-1.363149,0.268926,0.724114,1.474317,1.330155,2.171833,0.681011


In [12]:
y = np.concatenate((np.ones(len(condition)), np.zeros(len(control))))
y = pd.Series(y, dtype=int)

filepath = os.path.join(PROCESSED_DATA_DIR, f"depresjon_y.csv")
y.to_csv(filepath, header=False, index=False)

In [13]:
# to get ensemble:
import os

def sort_key(filename):
    parts = filename.split("_")
    # Wydobycie numerów jako liczb całkowitych
    condition_number = int(parts[1])  # Druga część np. "1" z "condition_1"
    segment_number = int(parts[3].split(".")[0])  # Trzecia część np. "1" z "segmen1.csv"
    return (condition_number, segment_number)


In [14]:
# Removing NaNs
folder_control = './data_night/depresjon/control'
folder_condition = './data_night/depresjon/condition'
z_control = []
z_condition = []
for filename in sorted(os.listdir(folder_control), key=sort_key):
    z_control.append(filename.split("_")[0] + filename.split("_")[1])
    
for filename in sorted(os.listdir(folder_condition), key=sort_key):
    z_condition.append(filename.split("_")[0] + filename.split("_")[1])
    
z_control = pd.Series(z_control)
z_condition = pd.Series(z_condition)

z = pd.concat([z_condition, z_control])
z.index = datasets["night"].index

mask = datasets["night"].notna().all(axis=1)
datasets["night"] = datasets["night"][mask]
y = y[mask]
z_depresjon = z[mask]

## Psykose

In [15]:
dataset = Dataset(dirpath=os.path.join("data_night", "psykose"))
condition = dataset.condition
control = dataset.control

In [16]:
condition_parts_dfs = extract_features_for_dataframes(condition, freq="H")
control_parts_dfs = extract_features_for_dataframes(control, freq="H")

datasets = {}

for part in ["night"]:
    condition_df = condition_parts_dfs[part]
    control_df = control_parts_dfs[part]

    entire_df = pd.concat([condition_df, control_df], ignore_index=True)

    # Przypisujemy wynik do słownika datasets
    datasets[part] = entire_df

In [17]:
for part, df in datasets.items():
    filename = f"{PSYKOSE_PREFIX}_{part}.csv"
    filepath = os.path.join(PROCESSED_DATA_DIR, filename)
    df.to_csv(filepath, index=False)

In [18]:
y = np.concatenate((np.ones(len(condition)), np.zeros(len(control))))
y = pd.Series(y, dtype=int)

filepath = os.path.join(PROCESSED_DATA_DIR, f"psykose_y.csv")
y.to_csv(filepath, header=False, index=False)
#datasets["full_24h"].dropna(inplace=True)

In [19]:
# Removing NaNs
folder_control = './data_night/psykose/control'
folder_condition = './data_night/psykose/condition'
z_control = []
z_condition = []
for filename in sorted(os.listdir(folder_control), key=sort_key):
    z_control.append(filename.split("_")[0] + filename.split("_")[1])
    
for filename in sorted(os.listdir(folder_condition), key=sort_key):
    z_condition.append(filename.split("_")[0] + filename.split("_")[1])
    
z_control = pd.Series(z_control)
z_condition = pd.Series(z_condition)

z = pd.concat([z_condition, z_control])
z.index = datasets["night"].index
mask = datasets["night"].notna().all(axis=1)
datasets["night"] = datasets["night"][mask]
y = y[mask]
z_psykose = z[mask]

Hyperactive

In [20]:
dataset = Dataset(dirpath=os.path.join("data_night", "hyperaktiv"))
condition = dataset.condition
control = dataset.control

In [21]:
condition_parts_dfs = extract_features_for_dataframes(condition, freq="H")
control_parts_dfs = extract_features_for_dataframes(control, freq="H")

datasets = {}

for part in ["night"]:
    condition_df = condition_parts_dfs[part]
    control_df = control_parts_dfs[part]

    entire_df = pd.concat([condition_df, control_df], ignore_index=True)

    # Przypisujemy wynik do słownika datasets
    datasets[part] = entire_df

In [22]:
for part, df in datasets.items():
    filename = f"{HYPERAKTIV_PREFIX}_{part}.csv"
    filepath = os.path.join(PROCESSED_DATA_DIR, filename)
    df.to_csv(filepath, index=False)

In [23]:
y = np.concatenate((np.ones(len(condition)), np.zeros(len(control))))
y = pd.Series(y, dtype=int)

filepath = os.path.join(PROCESSED_DATA_DIR, f"hyperaktiv_y.csv")
y.to_csv(filepath, header=False, index=False)

In [24]:
datasets["night"].dropna(inplace=True)
datasets["night"]

,minimum_time,maximum_time,mean_time,median_time,variance_time,kurtosis_time,skewness_time,coeff_of_var_time,iqr_time,trimmed_mean_time,...,mean_freq,median_freq,variance_freq,kurtosis_freq,skewness_freq,coeff_of_var_freq,iqr_freq,trimmed_mean_freq,entropy_freq,spectral_flatness
0,6.016667,230.183334,91.416679,65.266670,7912.653320,-1.510052,0.450157,0.927768,176.150001,85.487038,...,116072.601562,109778.625000,2.681830e+09,-1.578465,0.134022,0.446155,90220.271484,116072.593750,2.436998,0.894241
1,0.000000,73.683334,24.281818,16.816668,588.533997,-0.423892,0.953657,0.952595,30.108334,21.490740,...,12793.206055,10470.508789,1.164978e+08,-1.695204,0.203371,0.843683,20261.454712,12793.206055,1.983479,0.438438
2,0.000000,131.483337,42.448486,27.299999,2579.278076,-0.948834,0.820242,1.140750,70.308334,37.272224,...,48747.886719,56377.863281,9.152884e+08,-1.230018,-0.380039,0.620616,42063.210938,48747.890625,2.208131,0.535190
3,0.000000,250.916672,57.416664,16.816668,5966.505371,1.531150,1.564291,1.282702,77.891667,42.296295,...,37182.328125,37570.835938,4.365628e+08,-1.241227,-0.135190,0.561936,30318.465332,37182.332031,2.315747,0.757898
4,4.283333,166.449997,42.993935,16.766666,2538.044434,1.227072,1.483795,1.117239,50.774999,33.577774,...,68386.632812,43126.476562,3.511676e+09,-1.304017,0.637380,0.866535,89065.209961,68386.632812,2.046365,0.625485
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
567,6.650000,298.216675,90.040909,61.849998,8973.646484,0.482736,1.356871,1.003109,62.099999,76.175926,...,59718.839844,26482.294922,4.067366e+09,-1.308131,0.701559,1.067936,98704.467773,59718.839844,1.760489,0.440584
568,5.550000,513.766663,88.033333,41.150002,21160.406250,5.111922,2.553533,1.575500,74.858335,49.894444,...,103515.695312,25824.949219,1.592926e+10,-1.049457,0.854721,1.219247,159958.384766,103515.695312,1.541633,0.284065
569,5.216667,197.933334,65.021217,36.383335,4138.562500,-0.087868,1.107714,0.943351,59.875000,56.898151,...,52158.675781,23151.261719,3.426864e+09,0.262236,1.349471,1.122333,44557.569580,52158.675781,1.827806,0.558175
570,5.033333,426.116669,91.619698,28.716667,16389.708984,2.290914,1.819472,1.332295,97.125001,64.074074,...,111410.710938,11569.268555,2.198547e+10,-1.314444,0.771620,1.330886,208656.183105,111410.726562,1.297107,0.216173


# Classification

## Classifiers, parameters, constants

In [25]:
classifiers = {
    "LR": LogisticRegression(
        penalty="elasticnet",
        random_state=0,
        solver="saga",
        max_iter=500
    ),
    "SVM": SVC(
        kernel="rbf",
        cache_size=512
    ),
    "RF": RandomForestClassifier(
        n_estimators=200,
        criterion="entropy"
    ),
    "LGBM": LGBMClassifier(
        n_estimators=200,
        verbosity=-1,
        random_state=0
    ),
    "XGB": XGBClassifier(
        n_estimators=200,
        random_state=0 
    )
}


param_grids = {
    "LR": {
        "C": [0.001, 0.01, 0.1, 0.5, 1, 2, 5, 10, 25, 50, 100, 500, 1000],
        "class_weight": ["balanced"],
        "l1_ratio": [0, 0.05, 0.1, 0.15, 0.2, 0.25, 0.3, 0.35, 0.4, 0.45, 0.5,
                     0.55, 0.6, 0.65, 0.7, 0.75, 0.8, 0.85, 0.9, 0.95, 1],
        "max_iter": [1000]
    },
    "SVM": {
        "C": [1, 10],
        "gamma": ["scale"],
        "class_weight": [None, "balanced"]
    },
    "RF": {
        "class_weight": [None, "balanced", "balanced_subsample"]
    },
    "LGBM": {
        "num_leaves": [31, 50],
        "min_child_samples": [10, 20],
        "class_weight": [None, "balanced"]
    },
    "XGB": {
        "learning_rate": [0.01, 0.1],
        "max_depth": [3, 6],
        "subsample": [0.8, 1.0],
        "colsample_bytree": [0.8, 1.0],
        "scale_pos_weight": [1, 10]
    }
}

## Classification

# Classification - Depresjon

In [26]:
dataset = DEPRESJON_PREFIX  # DEPRESJON_PREFIX or PSYKOSE_PREFIX
y_filename = "depresjon_y.csv" if dataset == DEPRESJON_PREFIX else "psykose_y.csv"

In [27]:
# Assuming PROCESSED_DATA_DIR and y_filename are already defined
datasets = {}

# Load datasets
for part in ["night"]:
    filename = f"{dataset}_{part}.csv"
    filepath = os.path.join(PROCESSED_DATA_DIR, filename)
    datasets[part] = pd.read_csv(filepath, header=0).values

# Load y values
y = pd.read_csv(os.path.join(PROCESSED_DATA_DIR, y_filename), header=None, dtype=int)
y = y.values.ravel()
# Usuwanie wierszy, które zawierają NaN w danych
for part in datasets:
    # Indeksy wierszy, które zawierają NaN w dowolnej kolumnie
    nan_indices = np.isnan(datasets[part]).any(axis=1)
    
    # Usuwamy te wiersze z datasets i y
    datasets[part] = datasets[part][~nan_indices]
    y = y[~nan_indices]

# Sprawdzamy kształt danych po usunięciu NaN
#print(datasets['full_24h'].shape)
#print(y.shape)
#print(z_depresjon.shape)

In [28]:
print("Wyniki dla Depresjon:")
for part in ["night"]:
    print(f"PART: {part}")
    
    X = datasets[part]
    for clf_type in ["LR", "SVM", "RF", "LGBM", "XGB"]:
        print(f"  {clf_type}")
        folds = StratifiedKFold(n_splits=5, shuffle=True, random_state=0)
        
        test_scores = []
        group_results = {}  
        
        for train_idx, test_idx in folds.split(X, y):
            X_train, X_test = X[train_idx], X[test_idx] 
            y_train, y_test = y[train_idx], y[test_idx]
            
            X_train, X_test = variance_thresholding(X_train, X_test, threshold=0.05)
            X_train, X_test = standardize(X_train, X_test)
            
            grid_search = GridSearchCV(
                estimator=classifiers[clf_type], 
                param_grid=param_grids[clf_type], 
                scoring="f1_weighted",
                n_jobs=-1,
                refit=True,
                #cv=LeaveOneOut()
            )
            grid_search.fit(X_train, y_train)

            clf = grid_search.best_estimator_
            clf.fit(X_train, y_train)
            y_pred = clf.predict(X_test)
            for i in range(len(test_idx)):
                
                condition_id = z_depresjon.iloc[test_idx[i]] 
                class_label = y[test_idx[i]] 
                unique_id = (condition_id, class_label)  

                if unique_id not in group_results:
                    group_results[unique_id] = []  
                group_results[unique_id].append(y_pred[i])  
            final_predictions = {}
            for unique_id, preds in group_results.items():
                condition_id, class_label = unique_id
                majority_vote = mode(preds).mode[0]
                final_predictions[unique_id] = majority_vote 
            y_true_grouped = []
            y_pred_grouped =  []
            for k,v in final_predictions.items():
                y_true_grouped.append(k[1])
                y_pred_grouped.append(v)
            metrics = calculate_metrics(y_true_grouped, y_pred_grouped)
            test_scores.append(metrics)
            
        final_scores = calculate_metrics_statistics(test_scores)
        
        for metric, (mean, stddev) in final_scores.items():
            print(f"    {metric}: {mean:.4f} +- {stddev:.4f}")
        
        print()

Wyniki dla Depresjon:
PART: night
  LR
    accuracy: 0.6075 +- 0.0465
    balanced_accuracy: 0.5938 +- 0.0427
    f1: 0.5208 +- 0.0415
    precision: 0.5320 +- 0.0675
    recall: 0.5126 +- 0.0182
    specificity: 0.6750 +- 0.0702
    ROC_AUC: 0.5938 +- 0.0427
    MCC: 0.1903 +- 0.0884

  SVM
    accuracy: 0.6258 +- 0.0632
    balanced_accuracy: 0.6253 +- 0.0666
    f1: 0.5791 +- 0.0805
    precision: 0.5404 +- 0.0743
    recall: 0.6257 +- 0.0943
    specificity: 0.6250 +- 0.0625
    ROC_AUC: 0.6253 +- 0.0666
    MCC: 0.2480 +- 0.1316

  RF
    accuracy: 0.5535 +- 0.0261
    balanced_accuracy: 0.4907 +- 0.0246
    f1: 0.1818 +- 0.0845
    precision: 0.3567 +- 0.0998
    recall: 0.1251 +- 0.0669
    specificity: 0.8562 +- 0.0375
    ROC_AUC: 0.4907 +- 0.0246
    MCC: -0.0345 +- 0.0760

  LGBM
    accuracy: 0.4868 +- 0.0276
    balanced_accuracy: 0.4425 +- 0.0311
    f1: 0.2294 +- 0.0474
    precision: 0.3041 +- 0.0605
    recall: 0.1851 +- 0.0398
    specificity: 0.7000 +- 0.0424
    ROC

# Classification - Psykose

In [29]:
dataset = PSYKOSE_PREFIX  # DEPRESJON_PREFIX or PSYKOSE_PREFIX
y_filename = "depresjon_y.csv" if dataset == DEPRESJON_PREFIX else "psykose_y.csv"

In [30]:
for part in ["night"]:
    filename = f"{dataset}_{part}.csv"
    filepath = os.path.join(PROCESSED_DATA_DIR, filename)
    datasets[part] = pd.read_csv(filepath, header=0).values

# Load y values
y = pd.read_csv(os.path.join(PROCESSED_DATA_DIR, y_filename), header=None, dtype=int)
y = y.values.ravel()
# Usuwanie wierszy, które zawierają NaN w danych
for part in datasets:
    # Indeksy wierszy, które zawierają NaN w dowolnej kolumnie
    nan_indices = np.isnan(datasets[part]).any(axis=1)
    
    # Usuwamy te wiersze z datasets i y
    datasets[part] = datasets[part][~nan_indices]
    y = y[~nan_indices]

In [ ]:
print("Wyniki dla Psykose:")
for part in ["night"]:
    print(f"PART: {part}")
    
    X = datasets[part]
    for clf_type in ["LR", "SVM", "RF", "LGBM", "XGB"]:
        print(f"  {clf_type}")
        folds = StratifiedKFold(n_splits=5, shuffle=True, random_state=0)
        
        test_scores = []
        group_results = {}  
        
        for train_idx, test_idx in folds.split(X, y):
            X_train, X_test = X[train_idx], X[test_idx] 
            y_train, y_test = y[train_idx], y[test_idx]
            
            X_train, X_test = variance_thresholding(X_train, X_test, threshold=0.05)
            X_train, X_test = standardize(X_train, X_test)
            
            grid_search = GridSearchCV(
                estimator=classifiers[clf_type], 
                param_grid=param_grids[clf_type], 
                scoring="f1_weighted",
                n_jobs=-1,
                refit=True,
                #cv=LeaveOneOut()
            )
            grid_search.fit(X_train, y_train)

            clf = grid_search.best_estimator_
            clf.fit(X_train, y_train)
            y_pred = clf.predict(X_test)
            for i in range(len(test_idx)):
                
                condition_id = z_psykose.iloc[test_idx[i]] 
                class_label = y[test_idx[i]] 
                unique_id = (condition_id, class_label)  

                if unique_id not in group_results:
                    group_results[unique_id] = []  
                group_results[unique_id].append(y_pred[i])  
            final_predictions = {}
            for unique_id, preds in group_results.items():
                condition_id, class_label = unique_id
                majority_vote = mode(preds).mode[0]
                final_predictions[unique_id] = majority_vote 
            y_true_grouped = []
            y_pred_grouped =  []
            for k,v in final_predictions.items():
                y_true_grouped.append(k[1])
                y_pred_grouped.append(v)
            metrics = calculate_metrics(y_true_grouped, y_pred_grouped)
            test_scores.append(metrics)
            
        final_scores = calculate_metrics_statistics(test_scores)
        
        for metric, (mean, stddev) in final_scores.items():
            print(f"    {metric}: {mean:.4f} +- {stddev:.4f}")
        
        print()

Wyniki dla Psykose:
PART: night
  LR
    accuracy: 0.6852 +- 0.0586
    balanced_accuracy: 0.6918 +- 0.0644
    f1: 0.6479 +- 0.0847
    precision: 0.5933 +- 0.0555
    recall: 0.7273 +- 0.1379
    specificity: 0.6562 +- 0.0884
    ROC_AUC: 0.6918 +- 0.0644
    MCC: 0.3810 +- 0.1221

  SVM
    accuracy: 0.7259 +- 0.0378
    balanced_accuracy: 0.6835 +- 0.0380
    f1: 0.5670 +- 0.0826
    precision: 0.8260 +- 0.1256
    recall: 0.4545 +- 0.1185
    specificity: 0.9125 +- 0.0996
    ROC_AUC: 0.6835 +- 0.0380
    MCC: 0.4413 +- 0.0867

  RF
    accuracy: 0.7148 +- 0.0222
    balanced_accuracy: 0.6557 +- 0.0215
    f1: 0.4894 +- 0.0402
    precision: 0.9205 +- 0.1080
    recall: 0.3364 +- 0.0364
    specificity: 0.9750 +- 0.0364
    ROC_AUC: 0.6557 +- 0.0215
    MCC: 0.4317 +- 0.0662

  LGBM


# Classification - Hyperactive

In [ ]:
dataset = HYPERAKTIV_PREFIX
y_filename = "hyperaktiv_y.csv"

In [ ]:
for part in ["night"]:
    filename = f"{dataset}_{part}.csv"
    filepath = os.path.join(PROCESSED_DATA_DIR, filename)
    datasets[part] = pd.read_csv(filepath, header=0).values

# Load y values
y = pd.read_csv(os.path.join(PROCESSED_DATA_DIR, y_filename), header=None, dtype=int)
y = y.values.ravel()
# Usuwanie wierszy, które zawierają NaN w danych
for part in datasets:
    # Indeksy wierszy, które zawierają NaN w dowolnej kolumnie
    nan_indices = np.isnan(datasets[part]).any(axis=1)
    
    # Usuwamy te wiersze z datasets i y
    datasets[part] = datasets[part][~nan_indices]
    y = y[~nan_indices]

In [ ]:
print("Wyniki dla Hyperaktiv:")
for part in ["night"]:
    print(f"PART: {part}")
    
    X = datasets[part]
    for clf_type in ["LR", "SVM", "RF", "LGBM", "XGB"]:
        print(f"  {clf_type}")
        folds = StratifiedKFold(n_splits=5, shuffle=True, random_state=0)
        
        test_scores = []
        group_results = {}  
        
        for train_idx, test_idx in folds.split(X, y):
            X_train, X_test = X[train_idx], X[test_idx] 
            y_train, y_test = y[train_idx], y[test_idx]
            
            #X_train, X_test = variance_thresholding(X_train, X_test, threshold=0.05)
            X_train, X_test = standardize(X_train, X_test)
            
            grid_search = GridSearchCV(
                estimator=classifiers[clf_type], 
                param_grid=param_grids[clf_type], 
                scoring="f1_weighted",
                n_jobs=-1,
                refit=True,
                #cv=LeaveOneOut()
            )
            grid_search.fit(X_train, y_train)

            clf = grid_search.best_estimator_
            clf.fit(X_train, y_train)
            y_pred = clf.predict(X_test)
            for i in range(len(test_idx)):
                
                condition_id = z_psykose.iloc[test_idx[i]] 
                class_label = y[test_idx[i]] 
                unique_id = (condition_id, class_label)  

                if unique_id not in group_results:
                    group_results[unique_id] = []  
                group_results[unique_id].append(y_pred[i])  
            final_predictions = {}
            for unique_id, preds in group_results.items():
                condition_id, class_label = unique_id
                majority_vote = mode(preds).mode[0]
                final_predictions[unique_id] = majority_vote 
            y_true_grouped = []
            y_pred_grouped =  []
            for k,v in final_predictions.items():
                y_true_grouped.append(k[1])
                y_pred_grouped.append(v)
            metrics = calculate_metrics(y_true_grouped, y_pred_grouped)
            test_scores.append(metrics)
            
        final_scores = calculate_metrics_statistics(test_scores)
        
        for metric, (mean, stddev) in final_scores.items():
            print(f"    {metric}: {mean:.4f} +- {stddev:.4f}")
        
        print()